<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan/heatonresearch_gan_kit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Software

Some software must be installed into Colab, for this notebook to work. We are specificially using these technologies:

* [Training Generative Adversarial Networks with Limited Data](https://arxiv.org/abs/2006.06676)
Tero Karras, Miika Aittala, Janne Hellsten, Samuli Laine, Jaakko Lehtinen, Timo Aila
* [One millisecond face alignment with an ensemble of regression trees](https://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Kazemi_One_Millisecond_Face_2014_CVPR_paper.pdf) Vahid Kazemi, Josephine Sullivan


In [1]:
!rm -r /content/stylegan2-ada-pytorch/

rm: cannot remove '/content/stylegan2-ada-pytorch/': No such file or directory


In [2]:
!git clone https://github.com/mgzzl/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 530, done.
remote: Total 530 (delta 0), reused 0 (delta 0), pack-reused 530
Receiving objects: 100% (530/530), 8.77 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [2]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


In [9]:
!mkdir pretrained
!curl --output pretrained/ffhq.pkl 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl'
!curl --output pretrained/dog.pkl 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqdog.pkl'
!curl --output pretrained/metfaces.pkl 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl'

mkdir: cannot create directory ‘pretrained’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  363M  100  363M    0     0   127M      0  0:00:02  0:00:02 --:--:--  127M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  347M  100  347M    0     0  33.0M      0  0:00:10  0:00:10 --:--:-- 41.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  363M  100  363M    0     0   128M      0  0:00:02  0:00:02 --:--:--  128M


In [3]:
import os
import sys
import torch

!pip install ninja
!pip install opensimplex

import dnnlib
import legacy
import PIL.Image
import numpy as np
import imageio
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import cv2
import numpy as np

def display_lowres(img):
  plt.imshow(img)
  plt.title('poster-gan')
  plt.show()

def seed2vec(g, seed):
  return np.random.RandomState(seed).randn(1, g.z_dim)

def load_vec(filename):
  result = np.load(filename)
  return result if 'w' not in result else result['w']
  
def save_vec(filename, lvec):
  np.save(filename,lvec)

def to_image(img):
  return PIL.Image.fromarray(img, 'RGB')

def render_gan(G, device, lvec, label=None, truncation_psi=1, noise_mode='const'):
  # noise mode 'const', 'random', 'none'
  z = torch.from_numpy(lvec).to(device)
  if label is None: label = torch.zeros([1, G.c_dim], device=device)
  print(lvec.shape)
  if len(lvec.shape) == 2:
    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
  else:
    img = G.synthesis(z, noise_mode=noise_mode)
  
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  return PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB')

# Load a Pretrained GAN

In [4]:
STYLEGAN2_PKL_URL = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
#STYLEGAN2_PKL_URL = "https://github.com/jeffheaton/pretrained-gan-fish/releases/download/1.0.0/fish-gan-2020-12-09.pkl"
#STYLEGAN2_PKL_URL = "https://github.com/jeffheaton/pretrained-gan-70s-scifi/releases/download/v1/70s-scifi-gan-2020-12-12.pkl"
#STYLEGAN2_PKL_URL = "https://github.com/jeffheaton/pretrained-merry-gan-mas/releases/download/v1/christmas-gan-2020-12-03.pkl"
#STYLEGAN2_PKL_URL = "https://github.com/jeffheaton/pretrained-gan-minecraft/releases/download/v1/minecraft-gan-2020-12-22.pkl"
#STYLEGAN2_PKL_URL = "./out/training-runs/posterGAN2/00000-..."

device = torch.device('cuda')
with dnnlib.util.open_url(STYLEGAN2_PKL_URL) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

# Generate Images and Videos from Seeds

## Generate GAN Images from Seeds

The following code generates images from the loaded Pretrained GAN.

### Presettings for Generating

In [ ]:
import random

# Presettings
seeds = random.sample(range(0, 999999), 5)
OUTDIR = "/out/generated_images"

# Create directory
try:
    os.makedirs(OUTDIR)
except OSError:
    pass

### Generate Images

In [ ]:
# Generate images.

for seed_idx, seed in enumerate(seeds):
  print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
  lv = seed2vec(G, seed)
  save_vec(f'vec{seed:04d}.npy', lv)
  img = render_gan(G, device, lv)
  img.save(f'{OUTDIR}/seed{seed:04d}.png') # Save it
  #display(img2) # Display hi-res
  display_lowres(img)

## Build the Video from generated Images

The following code builds a transition video between the latent vectors from generated images.

### Presettings

Some Variables for Process

In [ ]:
import os
from pathlib import Path
import random

# To create a list of random integer values:
NFT_SEEDS = random.sample(range(0, 999999), 30)

# create a list of chosen integer values: 
#NFT_SEEDS = [1, 20, 400, 9]

# presettings directory
network_dir = "./pretrained"
outdir = "./out/generated-poster-nfts/" # "/" nach Directory wichtig!
print(f"Outputdirectory: {outdir}\nNetworkdirectory: {network_dir}\nSEEDS: {NFT_SEEDS}")

# Presettings for Video
STEPS = 150 # Steps between generated Images
FPS = 30 # How much frames per seconde?
FREEZE_STEPS = 0 # Freezing Steps for first_img before morphing to sec_img

Outputdirectory: ./out/generated-poster-nfts/
Networkdirectory: ./pretrained
SEEDS: [790213, 815131, 204608, 323750, 678755, 841602, 838201, 437987, 283032, 494448, 889128, 354701, 437390, 781593, 679795, 587188, 103758, 360856, 72964, 727425, 78071, 717363, 224467, 106448, 503798, 723990, 232772, 455859, 114570, 414497]


### Build the Video (npy - auto)

Searching for generated npy files

In [ ]:
import os
import sys
import glob
import torch
import dnnlib
import legacy
import PIL.Image
import numpy as np
import imageio
from tqdm.notebook import tqdm

lvec_root_dir = outdir
print(F"root_dir: {lvec_root_dir}")
lvec_sub_dir = [os.path.join(lvec_root_dir, o) for o in os.listdir(lvec_root_dir)
                                            if os.path.isdir(os.path.join(lvec_root_dir, o))]
print(F"sub_dir: {lvec_sub_dir}")
for lv_sd in lvec_sub_dir:
    seed_dir = [os.path.join(lv_sd + "/", n + "/", file) for n in os.listdir(lv_sd)
                                                            if os.path.isdir(os.path.join(lv_sd, n))
                                                                for file in os.listdir(os.path.join(lv_sd, n))
                                                                    if file.endswith(".npy")]
    
    lvec_list=[]
    for s in range(len(seed_dir)):
        lvec_list.append(np.load(f'{seed_dir[s]}')) # setup latent-vector list for every founded latent vector inside of single seed
    #lvec_list.append(np.load(f'{seed_dir[0]}')) # add first latent-vector also to the last to create a loop
    print(seed_dir[:1])
    print(f"\nLoading {len(lvec_list)} .npy files for {lv_sd}")
    lvec = lvec_list

    network_pkl = STYLEGAN2_PKL_URL
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

    target_uint8 = np.array([1024,1024,3], dtype=np.uint8)

    video = imageio.get_writer(f'{lv_sd}/movie.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

    for i in range(len(lvec)-1):
        lvec1 = lvec[i]
        lvec2 = lvec[i+1]

        diff = lvec2 - lvec1
        step = diff / STEPS
        current = lvec1.copy()

        for j in tqdm(range(STEPS)):
            z = torch.from_numpy(current).to(device)
            label=None
            truncation_psi=1
            noise_mode='const'
            if label is None: label = torch.zeros([1, G.c_dim], device=device)
            synth_image = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
            synth_image = (synth_image + 1) * (255/2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

            repeat = FREEZE_STEPS if (j==0 and i==0) or (j==(STEPS-1) and i==(len(lvec)-2)) else 1
        
            for i in range(repeat):
                video.append_data(synth_image)
                current = current + step
    
    video.close()

# Convert Image to a GAN

First, we convert the source to a GAN latent vector.  This process will take several minutes.

## Load a Saved Latent Vector and Display

## Build the Video (npz)

Load projected_w.npz manually and build the video

### Presettings

In [ ]:
STEPS = 150
FPS = 30
FREEZE_STEPS = 30
network_pkl = {STYLEGAN2_PKL_URL}

### Build the Video

In [ ]:
import torch
import dnnlib
import legacy
import PIL.Image
import numpy as np
import imageio
from tqdm.notebook import tqdm

lvec = [
  np.load('/content/first/projected_w.npz')['w'],
  np.load('/content/seconde/projected_w.npz')['w']
]

device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

target_uint8 = np.array([1024,1024,3], dtype=np.uint8)

video = imageio.get_writer('/content/movie.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

for i in range(len(lvec)-1):
  lvec1 = lvec[i]
  lvec2 = lvec[i+1]

  diff = lvec2 - lvec1
  step = diff / STEPS
  current = lvec1.copy()

  for j in tqdm(range(STEPS)):
    z = torch.from_numpy(current).to(device)
    synth_image = G.synthesis(z, noise_mode='const')
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

    repeat = FREEZE_STEPS if (j==0 and i==0) or (j==(STEPS-1) and i==(len(lvec)-2)) else 1
    
    for i in range(repeat):
      video.append_data(synth_image)
    current = current + step


video.close()

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
lvec = load_vec('./projected_w.npz')
img = render_gan(G, device, lvec)
display_lowres(img)

In [ ]:
SOURCE_NAME = "source-target.png"
OUT = "./out/projected_img/"

cmd = f"python ./projector.py --seed 1 --save-video 0 --num-steps 1000 --outdir={OUT} --target={SOURCE_NAME} --network={STYLEGAN2_PKL_URL}"
!{cmd}

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt

img_gan_source = cv2.imread(os.path.join(OUT,'proj.png'))
img = cv2.cvtColor(img_gan_source, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('poster-gan')
plt.show()

# Generate NFT (Interpolation)
The following code builds the latent_vectors and prints image for every seed in complete training_process

## Presettings for Interpolation

In [ ]:
import os
from pathlib import Path
import random

# To create a list of random integer values:
NFT_SEEDS = random.sample(range(0, 999999), 30)

# create a list of chosen integer values: 
#NFT_SEEDS = [1, 20, 400, 9]

# presettings directory
network_dir = "./pretrained"
outdir = "./out/generated-poster-nfts/" # "/" nach Directory wichtig!
print(f"Outputdirectory: {outdir}\nNetworkdirectory: {network_dir}\nSEEDS: {NFT_SEEDS}")

# How many Seeds per PKL (training_step)?
max_seed = 10

## Generate Interpolation from Images for chosen training_step

The following code builds the interpolatio-video for every seeds


In [1]:
grouped_nfts = NFT_SEEDS.copy()
#For every network pkl in networkdir
for file in os.listdir(network_dir):
            if file.endswith(".pkl"):
              #Go through every group in grouped list
              for single_seed in grouped_nfts[:max_seed]:
                pkl = os.path.join(network_dir, file)
                pkl_name = Path(pkl).stem
                cmd = f"python ./generate.py --random_seed {single_seed} --outdir={outdir}seed-{single_seed}/{pkl_name} --network={pkl} --process=interpolation --interpolation=circularloop --diameter=800.00 --frames=240"
                print(cmd)
                #!{cmd}
                grouped_nfts.pop(0)

print(grouped_nfts)


NameError: ignored

# Download your Video

If you made it through all of these steps, you are now ready to download your video.

In [ ]:
from google.colab import files
files.download("/content/movie.mp4") 

# Finetune an Image

If you find a seed that you like, you can fine tune it by adjusting the latent vector directly.  First, choose the seed to finetune.

In [ ]:
START_SEED = 3172

current = seed2vec(Gs, START_SEED)

Next, generate display the current vector. You will return to this point for each iteration of the finetuning.

In [ ]:
init_random_state(Gs, 10)
img = generate_image(Gs, current, 1.0)

SCALE = 0.5
display_image(img, SCALE)

Choose an explore size, this is the number of differnt potential images that will be chosen by moving in 10 different directions.  Run this code once initially, and then again anytime you wish to change the 10 directions you are exploring.  You might change the 10 directions if you are no longer seeing improvements.

In [ ]:
EXPLORE_SIZE = 25

explore = []
for i in range(EXPLORE_SIZE):
  explore.append( np.random.rand(1, 512) - 0.5 )

In [ ]:
# Choose the direction to move.  Choose -1 for the initial iteration.   
MOVE_DIRECTION = -1
SCALE = 0.5

if MOVE_DIRECTION >=0:
  current = current + explore[MOVE_DIRECTION]

for i, mv in enumerate(explore):
  print(f"Direction {i}")
  init_random_state(Gs, 10)
  z = current + mv
  img = generate_image(Gs, z, 1.0)
  display_image(img, SCALE)